### Function Attributes

So far, we have been dealing with non-callable attributes. When attributes are actually functions, things behave differently.

In [1]:
class Person:
    def say_hello():
        print('Hello!')

In [2]:
Person.say_hello

<function __main__.Person.say_hello()>

In [3]:
type(Person.say_hello)

function

As we can see it is just a plain function, and be called as usual:

In [4]:
Person.say_hello()

Hello!


Now let's create an instance of that class:

In [10]:
p = Person()
p1 = Person()

In [11]:
print(hex(id(Person)))
print(hex(id(p)))
print(hex(id(p1)))

0x15dbb1e1790
0x15dbbc66730
0x15dbbc66f10


In [13]:
Person.say_hello

<function __main__.Person.say_hello()>

We know we can access class attributes via the instance, so we should also be able to access the function attribute in the same way:

In [14]:
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x0000015DBBC66730>>

In [15]:
type(p.say_hello)

method

Hmm, the type has changed from `function` to `method`, and the function representation states that it is a **bound method** of the **specific object** `p` we created (notice the memory address).

And if we try to call the function from the instance, here's what happens:

In [16]:
try:
    p.say_hello()
except Exception as ex:
    print(type(ex).__name__, ex)

TypeError say_hello() takes 0 positional arguments but 1 was given


In [17]:
# say_hello() does not exist in the module, it is in the class
say_hello()

NameError: name 'say_hello' is not defined

`method` is an actual type in Python, and, like functions, they are callables, but they have one distinguishing feature. They need to be bound to an object, and that object reference is passed to the underlying function.

Often when we define functions in a class and call them from the instance we need to know which **specific** instance was used to call the function. This allows us to interact with the **instance variables**.

To do this, Python will automatically transform an ordinary function defined in a class into a method when it is called from an instance of the class.

Further, it will "bind" the method to the instance - meaning that the instance ID will be passed as the **first** argument to the function being called.

It does this using **descriptors** which we'll come back to in detail later.

For now let's just explore this a bit more:

In [18]:
class Person:
    def say_hello(*args):
        print('say_hello args:', args)

In [19]:
Person.say_hello()

say_hello args: ()


As we can see, calling `say_hello` from the **class**, just calls the function (it is just a function).

But when we call it from an instance:

In [20]:
p = Person()
hex(id(p))

'0x15dbbd4b6a0'

In [21]:
p.say_hello()

say_hello args: (<__main__.Person object at 0x0000015DBBD4B6A0>,)


You can see that the object `p` was passed as an argument to the class function `say_hello`.

The obvious advantage is that we can now interact with instance attributes easily:

In [22]:
class Person:
    def set_name(instance_obj, new_name):
        instance_obj.name = new_name  # or setattr(instance_obj, 'name', new_name)        

In [23]:
p = Person()

In [24]:
p.set_name('Alex')

In [25]:
p.__dict__

{'name': 'Alex'}

In [26]:
Person.name

AttributeError: type object 'Person' has no attribute 'name'

In [27]:
p.name

'Alex'

This has essentially the same effect as doing this:

In [28]:
Person.set_name(p, 'John')

In [29]:
p.__dict__

{'name': 'John'}

By convention, the first argument is usually named `self`, but ask you just saw we can name it whatever we want - it just will be in the instance when the method variant of the function is called - and it is called an **instance method**.

But **methods** are objects created by Python when calling class functions from an instance.

They have their own unique attributes too:

In [31]:
class Person:
    def set_name(self, new_name):
        self.name = new_name  # or setattr(instance_obj, 'name', new_name) 
    def say_hello(self):
        print(f'{self} says hello')

In [ ]:
#class Person:
#    def say_hello(self):
#        print(f'{self} says hello')

In [32]:
p = Person()
print(hex(id(p)))

0x15dbbd4b580


In [33]:
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x0000015DBBD4B580>>

In [34]:
p.say_hello()

<__main__.Person object at 0x0000015DBBD4B580> says hello


In [35]:
m_hello = p.say_hello

In [36]:
type(m_hello)

method

For example it has a `__func__` attribute:

In [37]:
m_hello.__func__

<function __main__.Person.say_hello(self)>

which happens to be the class function used to create the method (the underlying function).

But remember that a method is bound to an instance. In this case we got the method from the `p` object:

In [38]:
hex(id(p))

'0x15dbbd4b580'

In [39]:
m_hello.__self__  # This gives the id of the instance that the method is bound to

In [40]:
p.say_hello.__self__

As you can see, the method also has a reference to the object it is **bound** to.

So think of methods as functions that have been bound to a specific object, and that object is passed in as the first argument of the function call. The remaining arguments are then passed after that.

Instance methods are created automatically for us, when we define functions inside our class definitions.

This even holds true if we monkey-patch our classes at run-time:

An aside!  
In Python, the term monkey patch refers to dynamic (or run-time) modifications of a class or module. In Python, we can actually change the behavior of code at run-time.

In [41]:
class Person:
    def say_hello(self):
        print(f'instance method called from {self}')

In [42]:
p = Person()
hex(id(p))

'0x15dbbd4b700'

In [43]:
p.say_hello()

instance method called from <__main__.Person object at 0x0000015DBBD4B700>


In [44]:
Person.say_hello()

TypeError: say_hello() missing 1 required positional argument: 'self'

In [45]:
q = Person()
hex(id(q))

'0x15dbbd4b400'

In [46]:
q.say_hello()

instance method called from <__main__.Person object at 0x0000015DBBD4B400>


In [47]:
Person.do_work = lambda self: f"do_work called from {self}"

In [48]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'say_hello': <function __main__.Person.say_hello(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'do_work': <function __main__.<lambda>(self)>})

OK, so both functions are in the class `__dict__`.

let's create an instance and see what happens:

In [49]:
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x0000015DBBD4B700>>

In [50]:
p.do_work

<bound method <lambda> of <__main__.Person object at 0x0000015DBBD4B700>>

In [51]:
p.do_work()

'do_work called from <__main__.Person object at 0x0000015DBBD4B700>'

But be careful, if we add a function to the **instance** directly, this does not work the same - we have created a function in the instance, so it is not considered a method (since it was not defined in the class):

In [52]:
p.other_func = lambda *args: print(f'other_func called with {args}')

In [53]:
p.other_func

<function __main__.<lambda>(*args)>

In [54]:
'other_func' in Person.__dict__

False

In [55]:
'other_func' in p.__dict__

True

In [56]:
p.other_func()

other_func called with ()


As you can see, `other_func` is, and behaves, like an ordinary function.

Long story short, functions defined in a class are transformed into methods when called from instances of the class. So of course, we have to account for that extra argument that is passed to the method.